In [43]:
import random
import numpy as np
import tensorflow as tf

## Tetris engine

In [237]:
class Tetris():
    
    def __init__(self, width, height):
        
        self.width = width
        self.height = height
        self.grid = np.append(np.zeros(width*height), -np.ones(width))
        self.score = 0
        
        self.current_tetromino = None
        self.current_position = None
        self.current_rotation = None
        
        lTetromino = [
                [1,width+1,width*2+1,2],
                [width, width+1,width+2,width*2+2],
                [1,width+1,width*2+1,width*2],
                [0,width, width+1,width+2]
            ]
    
        zTetromino = [
                [1,2,width,width+1],
                [0,width,width+1,width*2+1],
                [1,2,width,width+1],
                [0,width,width+1,width*2+1]
            ]

        tTetromino = [
                [1,width,width+1,width+2],
                [1,width+1,width+2,width*2+1],
                [width,width+1,width+2,width*2+1],
                [1,width,width+1,width*2+1]
            ]

        oTetromino = [
                [0,1,width,width+1],
                [0,1,width,width+1],
                [0,1,width,width+1],
                [0,1,width,width+1]
            ]

        iTetromino = [
                [1,width+1,width*2+1,width*3+1],
                [width,width+1,width+2,width+3],
                [1,width+1,width*2+1,width*3+1],
                [width,width+1,width+2,width+3]
            ]

        self.theTetrominoes = [lTetromino, zTetromino, tTetromino, oTetromino,iTetromino]
        
        self.theTetrominoSizes = np.max(np.array(self.theTetrominoes) % self.width, axis=(1,2))+1
        
        self.status = "ONGOING"
        
    
    def start_new_tetromino(self):
        
        self.current_tetromino = np.random.choice(range(len(self.theTetrominoes)))
        self.current_position = 4
        self.current_rotation = 0
        
        #self.print_game()
    
    
    def print_game(self):
        
        grid = self.grid.copy()
        
        print( self.current_game(grid, self.current_tetromino, self.current_position, self.current_rotation).reshape(game.height+1, game.width) )
        
    
    def current_game(self, grid, tetromino_idx, position, rotation):
        
        grid = self.grid.copy()        
        tetromino = self.theTetrominoes[tetromino_idx][rotation]
        for pos in tetromino:
            grid[pos + position] = 2
        
        return grid
        
        
    def move(self, action):
        
        reward = 0
        
        if self.check_tetromino_and_stop():
            self.start_new_tetromino()
            
        if action==0:
            # do nothing
            pass
        elif action==1:
            # rotate
            #print("rotate")
            self.current_rotation, reward = self.rotate()
        elif action==2:
            # go left
            #print("left")
            self.current_position, reward = self.move_left()
        elif action==3:
            # go right
            #print("right")
            self.current_position, reward = self.move_right()
        elif action==4:
            # go down
            #print("down")
            self.current_position += self.width
        else:
            print("Invalid action")
        
        if self.check_tetromino_and_stop():
            self.start_new_tetromino()
        else:
            self.current_position += self.width
            if self.check_tetromino_and_stop():
                self.start_new_tetromino()

        self.grid, last_reward = self.check_lines_and_update_score(self.grid)
        reward += last_reward
        #print(f"Reward: {reward}")
        return reward
    
    
    def nextGrids(self):
        
        
        grids = []
            
        for m in range(5):
            
            grid = self.grid.copy()
            tetromino = self.current_tetromino
            position = self.current_position
            rotation = self.current_rotation
            
            if m==0:
                # do nothing
                pass
            elif m==1:
                # rotate
                rotation, reward = self.rotate()
            elif m==2:
                # go left
                position, reward = self.move_left()
            elif m==3:
                # go right
                position, reward = self.move_right()
            elif m==4:
                # go down
                position += self.width
            else:
                print("Invalid action")
            
            grid, tetromino, position, rotation = self.check_tetromino(grid, tetromino, position, rotation)
            position += self.width
            grid, tetromino, position, rotation = self.check_tetromino(grid, tetromino, position, rotation)
            grid = self.current_game(grid, tetromino, position, rotation)
            
            grids.append(grid)
            
        return grids
    

    def rotate(self):
        
        #tetromino = self.theTetrominoes[self.current_tetromino][self.current_rotation]
        rotation = (self.current_rotation + 1) %4
        rotated_tetromino = self.theTetrominoes[self.current_tetromino][rotation]
        width = self.width
        
        tetromino_size = self.theTetrominoSizes[self.current_tetromino]
        
        for pos in rotated_tetromino:
            if self.grid[pos+self.current_position] in [-1,1]:
                return self.current_rotation, 0
        
        if tetromino_size==3:
            if (self.current_position+1-1) % width==0 or (self.current_position+1+1) % width==0:
                return self.current_rotation, 0
        if tetromino_size==4 and self.current_rotation in [0,2]:
            if (self.current_position+1-1) % width==0 or (self.current_position+2+1) % width==0:
                return self.current_rotation, 0
        
        return rotation, 0
        
        
    def move_left(self):
        
        tetromino = self.theTetrominoes[self.current_tetromino][self.current_rotation]
        position = self.current_position - 1
        width = self.width
        
        for pos in tetromino:
            if (pos+self.current_position)//width!=(pos+position)//width or self.grid[pos+position]==1:
                #print(self.current_position, position, pos)
                #self.print_game()
                return self.current_position, 0
        
        return position, 0
    
    
    def move_right(self):
        
        tetromino = self.theTetrominoes[self.current_tetromino][self.current_rotation]
        position = self.current_position + 1
        width = self.width
        
        for pos in tetromino:
            if (pos+self.current_position)//width!=(pos+position)//width or self.grid[pos+position]==1:
                #print(self.current_position, position, pos)
                #self.print_game()
                return self.current_position, 0
        
        return position, 0
    
    
    def check_tetromino_and_stop(self):
        
        tetromino = self.theTetrominoes[self.current_tetromino][self.current_rotation]
        for pos in tetromino:
            if self.grid[pos + self.current_position + self.width] in [-1, 1]:
                for final_pos in tetromino:
                    self.grid[final_pos + self.current_position] = 1
                return True
        
        return False
    
    
    def check_tetromino(self, grid, tetromino_idx, position, rotation):
        
        tetromino = self.theTetrominoes[tetromino_idx][rotation]
        for pos in tetromino:
            if grid[pos + position + self.width] in [-1, 1]:
                for final_pos in tetromino:
                    grid[final_pos + position] = 1
                tetromino_idx = np.random.choice(range(len(self.theTetrominoes)))
                position = 4
                rotation = 0
                return grid, tetromino_idx, position, rotation
        
        return grid, tetromino_idx, position, rotation
    
    
    def check_lines_and_update_score(self, grid):
        
        score = 0
        
        for h in range(self.height):
            
            if (grid[h*self.width:(h+1)*self.width]==1).all():
                grid = np.delete(grid, range(h*self.width,(h+1)*self.width))
                self.score += 1
                grid = np.insert(grid, 0, np.zeros(self.width))
                #print(f"{h} line removed score = {self.score}")
                score += 1
            
        if (grid[:self.width]==1).any():
            self.status = "OVER"
            #print("Game Over")
            score = -1
    
        return grid, score
            

In [225]:
game = Tetris(width = 10, height = 20)
game.grid = np.append(np.ones(game.width*game.height), -np.ones(game.width))
game.grid.reshape(game.height+1, game.width)

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.

In [226]:
game.check_lines_and_update_score(game.grid)

(array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

In [227]:
game.score

20

In [228]:
game.grid.reshape(game.height+1, game.width)

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.

## Design DQN model

In [229]:
game = Tetris(width = 10, height = 20)
game.start_new_tetromino()

plays = 20

while plays>0:
    game.move(np.random.choice(range(5)))
    print("game")
    game.print_game()
    print("nexts")
    #[ print(g.reshape(21, 10)) for g in game.nextGrids()]
    plays -= 1

game.print_game()

game
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  2.  2.  2.  2.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]]
nexts
game
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.

In [230]:
activation_fn = tf.nn.leaky_relu

hidden_dimensions = 32
depth = 4

#inputs = tf.keras.layers.Input((game.width*game.height))
inputs = tf.constant(game.grid[:-game.width])

inputs = tf.reshape(inputs,  (-1,game.width, game.height) )
print(tf.shape(inputs))
one_hot_inputs = tf.one_hot(tf.cast(inputs, tf.int32),3)
print(tf.shape(one_hot_inputs))
    
#x = tf.keras.layers.MultiHeadAttention(num_heads=120, key_dim=120, attention_axes=1)(one_hot_inputs,one_hot_inputs)
x = tf.keras.layers.Conv2D(filters=81, kernel_size=4, activation=activation_fn)(one_hot_inputs)
print(tf.shape(x))
x = tf.keras.layers.Conv2D(filters=40, kernel_size=4, activation=activation_fn)(x)
print(tf.shape(x))
x = tf.keras.layers.Conv2D(filters=5, kernel_size=4, activation=activation_fn)(x)
print(tf.shape(x))
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(5, activation=tf.nn.sigmoid)(x)
print(tf.shape(x))

outputs = x

#model = tf.keras.Model(inputs=inputs, outputs=outputs, name="tetris_model")

tf.Tensor([ 1 10 20], shape=(3,), dtype=int32)
tf.Tensor([ 1 10 20  3], shape=(4,), dtype=int32)
tf.Tensor([ 1  7 17 81], shape=(4,), dtype=int32)
tf.Tensor([ 1  4 14 40], shape=(4,), dtype=int32)
tf.Tensor([ 1  1 11  5], shape=(4,), dtype=int32)
tf.Tensor([1 5], shape=(2,), dtype=int32)


In [231]:
def create_Q(width, height):
    
    activation_fn = tf.nn.leaky_relu

    hidden_dimensions = 32
    depth = 4

    inputs = tf.keras.layers.Input((width*height))
    
    x = tf.reshape(inputs,  (-1,game.width, game.height) )

    one_hot_inputs = tf.one_hot(tf.cast(x, tf.int32),3)
    
    x = tf.keras.layers.Conv2D(filters=10, kernel_size=4, activation=activation_fn)(one_hot_inputs)

    x = tf.keras.layers.Conv2D(filters=10, kernel_size=4, activation=activation_fn)(x)
    
    x = tf.keras.layers.Conv2D(filters=10, kernel_size=4, activation=activation_fn)(x)
    
    x = tf.keras.layers.Flatten()(x)
    
    x = tf.keras.layers.Dense(5)(x)

    outputs = x

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="tetris_model")

    return model


In [232]:
model = create_Q(10,20)

## Testing the tetromino for the rotation

In [233]:
game = Tetris(width = 10, height = 20)

In [234]:
sizes = np.max(np.array(game.theTetrominoes) % game.width, axis=(1,2)) +1
print(sizes)

[3 3 3 2 4]


## Training the AI

In [238]:
model = create_Q(10,20)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.Huber()

In [239]:
N = 10000
B = 10
epsilons = np.linspace(1.0, 0.3,N)
gamma = tf.constant(0.9)

for n in range(N):
        
    rewards = []
    next_max_rewards = []
    grids = []
    
    epsilon = epsilons[n]
    
    for m in range(B):
        game = Tetris(width = 10, height = 20)
        game.start_new_tetromino()
        while game.status!="OVER":

            if random.random()>epsilon:
                player = model(game.grid[:-game.width])
                move = tf.argmax(player, axis=1).numpy()[0]
                reward = game.move(move)
            else:
                reward = game.move(np.random.choice(range(5)))

            #print(game.nextGrids())
            next_grids = tf.concat([ g[:-game.width][np.newaxis,:] for g in game.nextGrids()], axis=0)
            next_max_reward = tf.reduce_max(model(next_grids)).numpy()
            next_max_rewards.append( next_max_reward )
            rewards.append(reward)
            grids.append(game.current_game(game.grid, game.current_tetromino, game.current_position, game.current_rotation)[:-game.width])


    training_grids = tf.concat([g[np.newaxis,:] for g in grids], axis=0)
    with tf.GradientTape() as tape:
        y = model(training_grids, training=True)
        y = tf.reduce_max(y, axis=1)
        loss_values = loss_fn(rewards + gamma*next_max_rewards, y)
        

    grads = tape.gradient(loss_values, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    lsv = round(loss_values.numpy(),4)
            
    print(f"Game round {n+1}: epsilon={round(epsilon,2)}, loss={lsv}, score={game.score}, max={max(rewards)}, min={min(rewards)}, length={len(rewards)}")
    print([(x,round(rewards.count(x)/len(rewards),2)) for x in set(rewards)])
        

Game round 1: epsilon=1.0, loss=0.006800000090152025, score=0, max=0, min=-1, length=1040
[(0, 0.99), (-1, 0.01)]
Game round 2: epsilon=1.0, loss=0.005400000140070915, score=0, max=0, min=-1, length=1246
[(0, 0.99), (-1, 0.01)]
Game round 3: epsilon=1.0, loss=0.00559999980032444, score=0, max=0, min=-1, length=1065
[(0, 0.99), (-1, 0.01)]
Game round 4: epsilon=1.0, loss=0.005100000184029341, score=0, max=0, min=-1, length=1118
[(0, 0.99), (-1, 0.01)]
Game round 5: epsilon=1.0, loss=0.004900000058114529, score=0, max=0, min=-1, length=1144
[(0, 0.99), (-1, 0.01)]
Game round 6: epsilon=1.0, loss=0.004800000227987766, score=0, max=0, min=-1, length=1190
[(0, 0.99), (-1, 0.01)]
Game round 7: epsilon=1.0, loss=0.004999999888241291, score=0, max=0, min=-1, length=1124
[(0, 0.99), (-1, 0.01)]
Game round 8: epsilon=1.0, loss=0.004900000058114529, score=0, max=0, min=-1, length=1119
[(0, 0.99), (-1, 0.01)]
Game round 9: epsilon=1.0, loss=0.004800000227987766, score=0, max=0, min=-1, length=1149

Game round 73: epsilon=0.99, loss=0.00430000014603138, score=0, max=0, min=-1, length=1178
[(0, 0.99), (-1, 0.01)]
Game round 74: epsilon=0.99, loss=0.004800000227987766, score=0, max=0, min=-1, length=1037
[(0, 0.99), (-1, 0.01)]
Game round 75: epsilon=0.99, loss=0.004600000102072954, score=0, max=0, min=-1, length=989
[(0, 0.99), (-1, 0.01)]
Game round 76: epsilon=0.99, loss=0.003700000001117587, score=0, max=0, min=-1, length=1176
[(0, 0.99), (-1, 0.01)]
Game round 77: epsilon=0.99, loss=0.00430000014603138, score=0, max=0, min=-1, length=975
[(0, 0.99), (-1, 0.01)]
Game round 78: epsilon=0.99, loss=0.004399999976158142, score=0, max=0, min=-1, length=1001
[(0, 0.99), (-1, 0.01)]
Game round 79: epsilon=0.99, loss=0.0034000000450760126, score=0, max=0, min=-1, length=1100
[(0, 0.99), (-1, 0.01)]
Game round 80: epsilon=0.99, loss=0.004000000189989805, score=0, max=0, min=-1, length=1104
[(0, 0.99), (-1, 0.01)]
Game round 81: epsilon=0.99, loss=0.00279999990016222, score=0, max=0, min=

Game round 144: epsilon=0.99, loss=0.004000000189989805, score=0, max=0, min=-1, length=1038
[(0, 0.99), (-1, 0.01)]
Game round 145: epsilon=0.99, loss=0.0031999999191612005, score=0, max=0, min=-1, length=1073
[(0, 0.99), (-1, 0.01)]
Game round 146: epsilon=0.99, loss=0.003100000089034438, score=0, max=0, min=-1, length=1136
[(0, 0.99), (-1, 0.01)]
Game round 147: epsilon=0.99, loss=0.005499999970197678, score=0, max=0, min=-1, length=1118
[(0, 0.99), (-1, 0.01)]
Game round 148: epsilon=0.99, loss=0.005200000014156103, score=0, max=0, min=-1, length=1147
[(0, 0.99), (-1, 0.01)]
Game round 149: epsilon=0.99, loss=0.0035000001080334187, score=0, max=0, min=-1, length=1112
[(0, 0.99), (-1, 0.01)]
Game round 150: epsilon=0.99, loss=0.002899999963119626, score=0, max=0, min=-1, length=1063
[(0, 0.99), (-1, 0.01)]
Game round 151: epsilon=0.99, loss=0.003700000001117587, score=0, max=0, min=-1, length=1036
[(0, 0.99), (-1, 0.01)]
Game round 152: epsilon=0.99, loss=0.004699999932199717, score

Game round 214: epsilon=0.99, loss=0.0034000000450760126, score=0, max=0, min=-1, length=1154
[(0, 0.99), (-1, 0.01)]
Game round 215: epsilon=0.99, loss=0.002300000051036477, score=0, max=0, min=-1, length=1078
[(0, 0.99), (-1, 0.01)]
Game round 216: epsilon=0.98, loss=0.003100000089034438, score=0, max=0, min=-1, length=973
[(0, 0.99), (-1, 0.01)]
Game round 217: epsilon=0.98, loss=0.0044999998062849045, score=0, max=0, min=-1, length=1128
[(0, 0.99), (-1, 0.01)]
Game round 218: epsilon=0.98, loss=0.005900000222027302, score=0, max=0, min=-1, length=1072
[(0, 0.99), (-1, 0.01)]
Game round 219: epsilon=0.98, loss=0.003800000064074993, score=0, max=0, min=-1, length=1065
[(0, 0.99), (-1, 0.01)]
Game round 220: epsilon=0.98, loss=0.003599999938160181, score=0, max=0, min=-1, length=1138
[(0, 0.99), (-1, 0.01)]
Game round 221: epsilon=0.98, loss=0.0026000000070780516, score=0, max=0, min=-1, length=959
[(0, 0.99), (-1, 0.01)]
Game round 222: epsilon=0.98, loss=0.0038999998942017555, score

Game round 284: epsilon=0.98, loss=0.006899999920278788, score=0, max=0, min=-1, length=916
[(0, 0.99), (-1, 0.01)]
Game round 285: epsilon=0.98, loss=0.003800000064074993, score=0, max=0, min=-1, length=1121
[(0, 0.99), (-1, 0.01)]
Game round 286: epsilon=0.98, loss=0.003700000001117587, score=0, max=0, min=-1, length=1031
[(0, 0.99), (-1, 0.01)]
Game round 287: epsilon=0.98, loss=0.0034000000450760126, score=0, max=0, min=-1, length=1088
[(0, 0.99), (-1, 0.01)]
Game round 288: epsilon=0.98, loss=0.002099999925121665, score=0, max=0, min=-1, length=1178
[(0, 0.99), (-1, 0.01)]
Game round 289: epsilon=0.98, loss=0.003599999938160181, score=0, max=0, min=-1, length=1017
[(0, 0.99), (-1, 0.01)]
Game round 290: epsilon=0.98, loss=0.0032999999821186066, score=0, max=0, min=-1, length=973
[(0, 0.99), (-1, 0.01)]
Game round 291: epsilon=0.98, loss=0.0026000000070780516, score=0, max=0, min=-1, length=966
[(0, 0.99), (-1, 0.01)]
Game round 292: epsilon=0.98, loss=0.002300000051036477, score=0

...

Game round 6663: epsilon=0.53, loss=0.003100000089034438, score=0, max=0, min=-1, length=725
[(0, 0.99), (-1, 0.01)]
Game round 6664: epsilon=0.53, loss=0.004800000227987766, score=0, max=0, min=-1, length=668
[(0, 0.99), (-1, 0.01)]
Game round 6665: epsilon=0.53, loss=0.0044999998062849045, score=0, max=0, min=-1, length=693
[(0, 0.99), (-1, 0.01)]
Game round 6666: epsilon=0.53, loss=0.003599999938160181, score=0, max=0, min=-1, length=665
[(0, 0.98), (-1, 0.02)]
Game round 6667: epsilon=0.53, loss=0.004999999888241291, score=0, max=0, min=-1, length=701
[(0, 0.99), (-1, 0.01)]
Game round 6668: epsilon=0.53, loss=0.00430000014603138, score=0, max=0, min=-1, length=647
[(0, 0.98), (-1, 0.02)]
Game round 6669: epsilon=0.53, loss=0.004699999932199717, score=0, max=0, min=-1, length=679
[(0, 0.99), (-1, 0.01)]
Game round 6670: epsilon=0.53, loss=0.0020000000949949026, score=0, max=0, min=-1, length=766
[(0, 0.99), (-1, 0.01)]
Game round 6671: epsilon=0.53, loss=0.003800000064074993, score

Game round 6733: epsilon=0.53, loss=0.004600000102072954, score=0, max=0, min=-1, length=739
[(0, 0.99), (-1, 0.01)]
Game round 6734: epsilon=0.53, loss=0.0034000000450760126, score=0, max=0, min=-1, length=714
[(0, 0.99), (-1, 0.01)]
Game round 6735: epsilon=0.53, loss=0.003800000064074993, score=0, max=0, min=-1, length=728
[(0, 0.99), (-1, 0.01)]
Game round 6736: epsilon=0.53, loss=0.00279999990016222, score=0, max=0, min=-1, length=714
[(0, 0.99), (-1, 0.01)]
Game round 6737: epsilon=0.53, loss=0.003700000001117587, score=0, max=0, min=-1, length=692
[(0, 0.99), (-1, 0.01)]
Game round 6738: epsilon=0.53, loss=0.005400000140070915, score=0, max=0, min=-1, length=713
[(0, 0.99), (-1, 0.01)]
Game round 6739: epsilon=0.53, loss=0.005499999970197678, score=0, max=0, min=-1, length=776
[(0, 0.99), (-1, 0.01)]
Game round 6740: epsilon=0.53, loss=0.00839999970048666, score=0, max=0, min=-1, length=732
[(0, 0.99), (-1, 0.01)]
Game round 6741: epsilon=0.53, loss=0.002300000051036477, score=0

Game round 6803: epsilon=0.52, loss=0.003800000064074993, score=0, max=0, min=-1, length=735
[(0, 0.99), (-1, 0.01)]
Game round 6804: epsilon=0.52, loss=0.004900000058114529, score=0, max=0, min=-1, length=665
[(0, 0.98), (-1, 0.02)]
Game round 6805: epsilon=0.52, loss=0.0071000000461936, score=0, max=0, min=-1, length=771
[(0, 0.99), (-1, 0.01)]
Game round 6806: epsilon=0.52, loss=0.0035000001080334187, score=0, max=0, min=-1, length=669
[(0, 0.99), (-1, 0.01)]
Game round 6807: epsilon=0.52, loss=0.00430000014603138, score=0, max=0, min=-1, length=659
[(0, 0.98), (-1, 0.02)]
Game round 6808: epsilon=0.52, loss=0.00279999990016222, score=0, max=0, min=-1, length=659
[(0, 0.98), (-1, 0.02)]
Game round 6809: epsilon=0.52, loss=0.004699999932199717, score=0, max=0, min=-1, length=709
[(0, 0.99), (-1, 0.01)]
Game round 6810: epsilon=0.52, loss=0.006599999964237213, score=0, max=0, min=-1, length=728
[(0, 0.99), (-1, 0.01)]
Game round 6811: epsilon=0.52, loss=0.006099999882280827, score=0, 

Game round 6873: epsilon=0.52, loss=0.004999999888241291, score=0, max=0, min=-1, length=672
[(0, 0.99), (-1, 0.01)]
Game round 6874: epsilon=0.52, loss=0.004600000102072954, score=0, max=0, min=-1, length=661
[(0, 0.98), (-1, 0.02)]
Game round 6875: epsilon=0.52, loss=0.004399999976158142, score=0, max=0, min=-1, length=727
[(0, 0.99), (-1, 0.01)]
Game round 6876: epsilon=0.52, loss=0.005499999970197678, score=0, max=0, min=-1, length=624
[(0, 0.98), (-1, 0.02)]
Game round 6877: epsilon=0.52, loss=0.00430000014603138, score=0, max=0, min=-1, length=810
[(0, 0.99), (-1, 0.01)]
Game round 6878: epsilon=0.52, loss=0.003000000026077032, score=0, max=0, min=-1, length=726
[(0, 0.99), (-1, 0.01)]
Game round 6879: epsilon=0.52, loss=0.005100000184029341, score=0, max=0, min=-1, length=646
[(0, 0.98), (-1, 0.02)]
Game round 6880: epsilon=0.52, loss=0.00559999980032444, score=0, max=0, min=-1, length=631
[(0, 0.98), (-1, 0.02)]
Game round 6881: epsilon=0.52, loss=0.004399999976158142, score=0,

KeyboardInterrupt: 

In [ ]:
game = Tetris(width = 10, height = 20)
game.start_new_tetromino()

plays = 20

while plays>0:
    player = model(game.grid[:-game.width])
    move = tf.argmax(player, axis=1).numpy()[0]
    game.move(move)
    #game.move(np.random.choice(range(5)))
    print("game")
    game.print_game()
    print(model(game.current_game(game.grid, game.current_tetromino, game.current_position, game.current_rotation)[:-game.width]))
    
    print("nexts")
    #[ print(g.reshape(21, 10)) for g in game.nextGrids()]
    plays -= 1

game.print_game()